<a href="https://colab.research.google.com/github/Sreerag-Pillai/Data_Science/blob/main/Trading_strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib pandas mplfinance plotly seaborn numpy scipy mplfinance yfinance

## 1. Support and Resistance


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
# Load the dataset
sample_data = pd.read_csv('/content/drive/MyDrive/SampleData.csv')

sample_data.head()

In [ ]:
import numpy as np
import pandas as pd
import mplfinance as mpf
import yfinance as yf

def fetch_data_from_yahoo(ticker_symbol, start_date, end_date):
    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
    return stock_data.reset_index()

def detect_pivot_points(data, window=3):
    # Initializing columns
    data['PivotHigh'] = np.nan
    data['PivotLow'] = np.nan

    for i in range(window, len(data) - window):
        current_point = data.iloc[i]

        # Check if it's a potential resistance (local maxima)
        if all(current_point['High'] >= data.iloc[i-window:i]['High']) and all(current_point['High'] >= data.iloc[i+1:i+window+1]['High']):
            data.loc[current_point.name, 'PivotHigh'] = current_point['High']

        # Check if it's a potential support (local minima)
        if all(current_point['Low'] <= data.iloc[i-window:i]['Low']) and all(current_point['Low'] <= data.iloc[i+1:i+window+1]['Low']):
          data.loc[current_point.name, 'PivotLow'] = current_point['Low']

    return data

def plot_support_resistance_breaks_mpf(df):
    mc = mpf.make_marketcolors(up='g', down='r', inherit=True)
    s = mpf.make_mpf_style(base_mpf_style="yahoo", marketcolors=mc)

    ap = [mpf.make_addplot(df['PivotHigh'], type='scatter', markersize=100, marker='^', color='red', ylabel='Resistance'),
          mpf.make_addplot(df['PivotLow'], type='scatter', markersize=100, marker='v', color='blue', ylabel='Support')]

    mpf.plot(df, type='candle', style=s, figsize=(15,7),
             title='Support & Resistance with Breaks',
             ylabel='Price',
             addplot=ap,
             volume=True)

if __name__ == '__main__':
    df = fetch_data_from_yahoo('TSLA', '2021-11-01', '2023-01-30')
    df.set_index('Date', inplace=True)
    df = detect_pivot_points(df, window=3)

    if not df[['PivotHigh', 'PivotLow']].isna().all().all():
        plot_support_resistance_breaks_mpf(df)
    else:
        print("No pivot points detected.")


## 2. Trendline

###     Date: September 2023


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import linregress

# Load the data
df = pd.read_csv('/content/drive/SampleData.csv')

# Concatenate 'Date' and 'Time' into a single datetime column
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), format='%Y%m%d %H:%M')

def pivotid(df1, l, n1, n2):
    if l - n1 < 0 or l + n2 >= len(df1):
        return 0

    pividlow = 1
    pividhigh = 1
    for i in range(l - n1, l + n2 + 1):
        if df1.Low[l] > df1.Low[i]:
            pividlow = 0
        if df1.High[l] < df1.High[i]:
            pividhigh = 0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

# Calculate pivot points for the entire dataset
df['pivot'] = df.apply(lambda x: pivotid(df, x.name, 3, 3), axis=1)

def pointpos(x):
    if x['pivot'] == 1:
        return x['Low'] - 1e-3
    elif x['pivot'] == 2:
        return x['High'] + 1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

# Extract the peaks and troughs
peaks = df[df['pivot'] == 2]
troughs = df[df['pivot'] == 1]

# Create a function to draw trendlines
def draw_trendlines(df, peaks, troughs):
    fig = go.Figure(data=[go.Candlestick(x=df['DateTime'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    # Draw upward trendlines
    for i in range(1, len(troughs)):
        if troughs['Low'].iloc[i] != troughs['Low'].iloc[i-1]:  # Check if two consecutive troughs are equal
            start_x = troughs['DateTime'].iloc[i-1]
            end_x = troughs['DateTime'].iloc[i]
            start_y = troughs['Low'].iloc[i-1]
            end_y = troughs['Low'].iloc[i]

            fig.add_trace(go.Scatter(
                x=[start_x, end_x],
                y=[start_y, end_y],
                mode='lines',
                line=dict(color='red', width=2),
                showlegend=False
            ))

    # Draw downward trendlines
    for i in range(1, len(peaks)):
        if peaks['High'].iloc[i] != peaks['High'].iloc[i-1]:  # Check if two consecutive peaks are equal
            start_x = peaks['DateTime'].iloc[i-1]
            end_x = peaks['DateTime'].iloc[i]
            start_y = peaks['High'].iloc[i-1]
            end_y = peaks['High'].iloc[i]

            fig.add_trace(go.Scatter(
                x=[start_x, end_x],
                y=[start_y, end_y],
                mode='lines',
                line=dict(color='green', width=2),
                showlegend=False
            ))

    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()

# Draw the trendlines on the candlestick chart
draw_trendlines(df, peaks, troughs)


## 3. Point and Figure Charting

###     Date: September 2023


source:https://github.com/cjknox/PyPnF

In [ ]:
sample_data.index = pd.to_datetime(sample_data.index)
box_size = 1

def compute_pf(data, box_size=1, reversal=3):
    prices = data['Close'].values
    direction = None
    current_level = prices[0]
    pf_data = []

    for price in prices[1:]:
        diff = price - current_level
        boxes = int(diff / box_size)

        if direction is None:
            if boxes > 0:
                direction = "X"
            elif boxes < 0:
                direction = "O"
            else:
                continue

        if direction == "X" and boxes > 0:
            for _ in range(boxes):
                current_level += box_size
                pf_data.append(('X', current_level))

        elif direction == "O" and boxes < 0:
            for _ in range(-boxes):
                current_level -= box_size
                pf_data.append(('O', current_level))

        elif direction == "X" and boxes < 0 and abs(boxes) >= reversal:
            direction = "O"
            current_level -= box_size * (abs(boxes) - reversal + 1)
            pf_data.append(('O', current_level))

        elif direction == "O" and boxes > 0 and boxes >= reversal:
            direction = "X"
            current_level += box_size * (boxes - reversal + 1)
            pf_data.append(('X', current_level))

    return pf_data

pf_data = compute_pf(sample_data)

fig, axes = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [2, 1]})

# Plotting candlestick on the top axis
mpf.plot(sample_data, type='candle', style='yahoo', ax=axes[0], figratio=(15,10))

# Plotting P&F on the bottom axis
col_number = 0
last_level = pf_data[0][1]  # Initialize with the first level in pf_data

for i in range(1, len(pf_data)):
    direction, level = pf_data[i]
    prev_direction, prev_level = pf_data[i-1]
    if direction != prev_direction:
        col_number += 1
        if direction == "X":
            last_level += box_size
        else:
            last_level -= box_size
    else:
        if direction == "X":
            last_level += box_size
        else:
            last_level -= box_size

    if direction == "X":
        axes[1].scatter([col_number], [last_level], marker="X", color="g")
    else:
        axes[1].scatter([col_number], [last_level], marker="o", color="r")

# axes[1].invert_yaxis()  # Commented out this line to fix the y-axis inversion
axes[1].set_title("Point & Figure Chart")
plt.tight_layout()
plt.show()



## 4. Triangle or Wedge

###     Date: September 2023


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import linregress

# Load the data
df = pd.read_csv('Dataset 2022 - 5 Minute Data.CSV')

# Concatenate 'Date' and 'Time' into a single datetime column
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str), format='%Y%m%d %H:%M')

def pivotid(df1, l, n1, n2):
    if l - n1 < 0 or l + n2 >= len(df1):
        return 0

    pividlow = 1
    pividhigh = 1
    for i in range(l - n1, l + n2 + 1):
        if df1.Low[l] > df1.Low[i]:
            pividlow = 0
        if df1.High[l] < df1.High[i]:
            pividhigh = 0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

# Calculate pivot points for the entire dataset
df['pivot'] = df.apply(lambda x: pivotid(df, x.name, 3, 3), axis=1)

def pointpos(x):
    if x['pivot'] == 1:
        return x['Low'] - 1e-3
    elif x['pivot'] == 2:
        return x['High'] + 1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

backcandles = 20
candleid = 2056

# Ensure candleid is within the valid range
if candleid - backcandles < 0:
    candleid = backcandles
if candleid + 1 >= len(df):
    candleid = len(df) - 2

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

for i in range(candleid - backcandles, candleid + 1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].Low)
        xxmin = np.append(xxmin, i)
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].High)
        xxmax = np.append(xxmax, i)

slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

print(rmin, rmax)

dfpl = df[candleid - backcandles - 10:candleid + backcandles + 10]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot")

xxmin = np.append(xxmin, xxmin[-1] + 15)
xxmax = np.append(xxmax, xxmax[-1] + 15)

fig.add_trace(go.Scatter(x=xxmin, y=slmin * xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax * xxmax + intercmax, mode='lines', name='max slope'))
fig.update_xaxes(type='category')  # Use category type for x-axis to ensure correct ordering
fig.show()

import numpy as np
from matplotlib import pyplot
from scipy.stats import linregress

backcandles= 20

for candleid in range(0, len(df)):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].Low)
            xxmin = np.append(xxmin, i)
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].High)
            xxmax = np.append(xxmax, i)

    if (xxmax.size <3 and xxmin.size <3) or xxmax.size==0 or xxmin.size==0:
        continue

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

    if abs(rmax) >= 0.7 and abs(rmin) >= 0.7 and 0 < slmin <= 0.05 and -0.05 <= slmax < 0:
        print(rmin, rmax, candleid)
        break

    if candleid % 1000 == 0:
        print(candleid)